In [1]:
import findspark
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
findspark.init()
findspark.find()

'C:\\tools\\spark-3.3.2-bin-hadoop3'

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
#We read our file
df_green = spark.read \
            .parquet("./data/pq/green/*/*")

In [6]:
df_green.registerTempTable('green')

In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-02 14:00:00|  26| 327.4800000000001|            11|
|2020-01-06 15:00:00| 241|             184.7|             3|
|2020-01-22 13:00:00|  74| 666.2399999999999|            44|
|2020-01-14 23:00:00| 223|             48.31|             4|
|2020-01-09 23:00:00| 260|202.33000000000004|            16|
|2020-01-30 09:00:00|  75|1010.3499999999995|            67|
|2020-01-11 21:00:00|  41| 700.5999999999997|            50|
|2020-01-07 18:00:00|  45|112.39000000000001|             4|
|2020-01-02 09:00:00|  42| 555.7500000000001|            33|
|2020-01-04 23:00:00|  25|             376.4|            26|
|2020-01-25 11:00:00| 259|            161.21|             3|
|2020-01-16 20:00:00|  52|             108.6|             7|
|2020-01-16 06:00:00|  74| 600.7999999999998|            37|
|2020-01-17 07:00:00|  4

In [9]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
#We read our file
df_yellow = spark.read \
            .parquet("./data/pq/yellow/*/*")

In [11]:
df_yellow.registerTempTable("yellow")

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-16 18:00:00| 232|261.71000000000004|            12|
|2020-01-06 05:00:00|  48|2105.5099999999984|           136|
|2020-01-03 15:00:00| 163| 5261.330000000002|           348|
|2020-01-06 11:00:00|  43|2731.4199999999983|           185|
|2020-01-17 07:00:00|  24|           1058.45|            62|
|2020-01-12 11:00:00| 163| 4039.869999999997|           244|
|2020-01-20 20:00:00| 144|1758.8999999999996|           107|
|2020-01-27 18:00:00| 162|11701.700000000033|           698|
|2020-01-26 03:00:00| 263|           1156.81|            69|
|2020-01-20 18:00:00| 113|2801.8799999999987|           197|
|2020-01-24 15:00:00| 186|7813.4200000000155|           446|
|2020-01-26 16:00:00| 236| 7387.040000000016|           509|
|2020-01-09 18:00:00| 263| 4200.289999999999|           268|
|2020-01-26 16:00:00| 23

In [14]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [15]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [16]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [17]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [18]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  33| 317.2700000000001|                  11|            255.56|                    8|
|2020-01-01 00:00:00|  62|             15.95|                   1|             61.43|                    1|
|2020-01-01 00:00:00|  71|              23.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  81|54.870000000000005|                   2|             30.32|                    1|
|2020-01-01 00:00:00| 125|              null|                null|           1342.07|                   68|
|2020-01-01 00:00:00| 126|  

In [19]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [20]:
df_join = spark.read.parquet('data/report/revenue/total')

In [21]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000002|                   45|
|2020-01-01 00:00:00|  37|175.67000000000002|                   6|            161.61|                    7|
|2020-01-01 00:00:00|  43|            107.52|                   6| 6539.510000000011|                  390|
|2020-01-01 00:00:00|  49|266.76000000000005|                  14|185.64999999999998|                   10|
|2020-01-01 00:00:00|  52| 8

In [22]:
df_zones = spark.read.option("header","true").csv("./taxi_zone_lookup.csv")

In [23]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [24]:

df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [25]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')